In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#reading the Emission current, pressures
vaclog=pd.read_csv("static vac log", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,15/11/2022 17:07:38,7.0,-0.065991,0.0,0.0,4.996000e-09,5.198000e-07,0.000485,226.0,-5.050,4307.727,NaN,NaN
2,NaN,15/11/2022 17:07:45,14.0,-0.065550,0.0,0.0,4.996000e-09,5.198000e-07,0.000470,226.0,-5.387,4307.727,NaN,NaN
3,NaN,15/11/2022 17:07:53,22.0,-0.066239,0.0,0.0,4.996000e-09,5.198000e-07,0.000486,226.0,-5.163,4307.839,NaN,NaN
4,NaN,15/11/2022 17:08:00,29.0,-0.066733,0.0,0.0,4.996000e-09,5.192000e-07,0.000512,226.0,-4.938,4307.727,NaN,NaN
5,NaN,15/11/2022 17:08:07,36.0,-0.066584,0.0,0.0,4.996000e-09,5.192000e-07,0.000489,226.0,-5.163,4307.839,NaN,NaN


In [3]:
#reading the Emission current, pressures
vaclog2=pd.read_csv("warmup", sep="\t")
vaclog2.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog2["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog2["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog2.insert(2,"Elapsed time",runtime)
vaclog2.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog2.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,17/11/2022 17:34:24,0.0,-0.062171,1.810000e-09,2.180000e-09,4.996000e-09,5.305000e-07,0.000482,204.0,-5.275,4325.572,NaN,NaN
2,NaN,17/11/2022 17:34:38,14.0,-0.062147,1.180000e-09,6.410000e-10,4.996000e-09,5.305000e-07,0.000474,204.0,-5.163,4325.684,NaN,NaN
3,NaN,17/11/2022 17:34:45,21.0,-0.061852,1.130000e-09,5.910000e-10,4.996000e-09,5.305000e-07,0.000453,204.0,-5.050,4325.796,NaN,NaN
4,NaN,17/11/2022 17:34:52,28.0,-0.062367,1.110000e-09,5.790000e-10,4.996000e-09,5.305000e-07,0.000445,203.0,-5.163,4325.684,NaN,NaN
5,NaN,17/11/2022 17:35:00,36.0,-0.062713,1.100000e-09,5.740000e-10,4.996000e-09,5.305000e-07,0.000449,204.0,-5.275,4325.796,NaN,NaN


In [4]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]/3600),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='Overnight warmup')
plt.plot((vaclog2["Elapsed time"]/3600),polyfit(A,vaclog2["T-CERNOX"]),marker=".", markersize=5,label='Test')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Overnight warmup temperature evolution")
plt.yscale('linear')